In [1]:
!which python

/Users/pushkalm11/miniconda3/envs/unrolling/bin/python


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power as MP

In [3]:
import random
import scipy
from scipy.spatial.distance import cdist
from scipy.io import savemat, loadmat

In [4]:
import networkx as nx
import sys

In [5]:
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Flatten, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping

In [6]:
print(f"TF Version: {tf.__version__}")

TF Version: 2.14.0


In [7]:
X_train = loadmat('./Dataset/X_train.mat')['X_Ground']
Adj_train = loadmat('./Dataset/Adj.mat')['A_Ground']

N, T = X_train.shape

In [8]:
dt = np.zeros([T - 1, T])

for i in range(T - 1):
    dt[i, i] = -1
    dt[i, i - 1] = 1

d = dt.T
d = np.concatenate((d, np.zeros([T, 1])), axis = 1) 
d[T - 1, T - 1] = -1
d = d.T
d[T - 1, T - 1] = 1 
d[-1, -2] = 1
d[0, -1] = 0
d = tf.convert_to_tensor(d, dtype = "float32")
print(d)

tf.Tensor(
[[-1.  0.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 [ 0.  1. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  0.  1.  1.]], shape=(500, 500), dtype=float32)


In [9]:
def adj_to_laplacian(AdjL):
    tmp = np.array(AdjL, dtype = "float32")
    return np.diag(np.sum(tmp, axis = 0)) - tmp

In [10]:
def initial_graph(X_training, N, top_values):
    # top_values = 6
    distance_matrix = cdist(X_training, X_training, metric = 'euclidean')
    sorted_indices = distance_matrix.argsort(axis = 1)
    
    TAdj = np.zeros([N, N])
    
    for i in range(N):
        for j in range(1, top_values):
            k = sorted_indices[i][j]
            TAdj[i][k] = 1
            
    TAdj = (TAdj + TAdj.T) / 2
    TAdj = np.where(TAdj > 0.1, 1, 0)
    AdjL = np.diag(np.sum(TAdj, 1)) - TAdj
    
    return AdjL

In [11]:
psi_cost = 0
test_list = []

In [12]:
# A - Predicted, B - Target matrix
def costfunc(A, B):
    # (y_true, y_pred)
    # print(A.shape)
    # print(B.shape)
    return (tf.norm(tf.multiply(psi_cost, (A - B)))) ** 2

# def costfunc(A, B):
#     return (tf.norm(tf.multiply(psi_cost, (A - B))) ** 2) + (reg * tf.norm(HLG @ B @ GLT) ** 2 / 2)

def mse_unknown(A, B):

    global psi_cost
    psi = psi_cost
    
    psid = tf.convert_to_tensor(np.ones([N, T], dtype = float) - psi, dtype = "float32")
    mse_unk = (LA.norm(np.multiply(psid, (X_train - B)))) ** 2 / np.sum(psid.numpy().flatten())
    
    global mse_unknown_list
    mse_unknown_list.append(mse_unk)
    
    return mse_unk

def mse_known(A, B):

    global psi_cost 
    psi = psi_cost

    mse_kn = (LA.norm(np.multiply(psi, (X_train - B)))) ** 2 / np.sum(psi.numpy().flatten())
    
    global mse_known_list
    mse_known_list.append(mse_kn)

    return mse_kn

In [13]:
callb = EarlyStopping(monitor = 'loss', mode = 'min', min_delta = 0.00001, verbose = 1, patience = 10)

def scheduler(epoch, lr):
    if epoch < 15:
        return lr
    else:
        return lr / lr_decay_factor

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [14]:
class EldarModel(tf.keras.Model):
    
    def __init__(self, Laplacian, reg_init, loops_init, d_init, **kwargs):
        super(EldarModel, self).__init__(**kwargs)
        p_init = 10.0
        self.p1_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p1 = tf.Variable(initial_value = self.p1_init, trainable = True, name = "p1")
        self.p2_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p2 = tf.Variable(initial_value = self.p2_init, trainable = True, name = "p2")
        self.p3_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p3 = tf.Variable(initial_value = self.p3_init, trainable = True, name = "p3")
        self.p4_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p4 = tf.Variable(initial_value = self.p4_init, trainable = True, name = "p4")
        
        self.lap = Laplacian
        self.reg = reg_init
        self.loops = loops_init
        self.d_matrix = d_init
        
    def call(self, y):
        
        psi = y[0, :, T:]
        global psi_cost
        psi_cost = psi
        
        psi = tf.convert_to_tensor(psi, dtype = "float32")
        
        X_in = y[0, :, : T]
        X_in = tf.convert_to_tensor(X_in, dtype = "float32")
        
        Y = X_in
        
        LG, reg, loop, d = self.lap, self.reg, self.loops, self.d_matrix
        
        LT = tf.matmul(tf.transpose(d), d)
        LT = tf.convert_to_tensor(LT, dtype = "float32")
        
        HLG = tf.eye(tf.shape(LG)[0], dtype = "float32") 
        GLT = LT + (self.p1 * MP(LT, 2)) + (self.p2 * MP(LT, 3)) + (self.p3 * MP(LT, 4)) + (self.p4 * MP(LT, 5))
        
        Xk = tf.zeros_like(X_in)
        Zk = tf.zeros_like(X_in)
        Zk = -(tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT)))
        
        for i in range(loop):
            fdx_xk = tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT))
            fdx_zk = tf.multiply(psi, Zk) - Y + (reg * (HLG @ Zk @ GLT))
            
            # a = (tf.multiply(psi, Xk) - Y).numpy()[:, 0]
            # b = (reg * (HLG @ Xk @ GLT)).numpy()[:, 0]
            # tmp = list(zip(a, b))
            # if i <= (loop - 1):
            #     for t in tmp:
            #         print(t)
            #     print("\n")
            
            tau = tf.linalg.trace(tf.transpose(fdx_xk) @ Zk) / tf.linalg.trace(tf.transpose((Y + fdx_zk)) @ Zk)
            
            Xk_1 = Xk - (tau * Zk)
            
            fdx_xk_1 = tf.multiply(psi, Xk_1) - Y + (reg * (HLG @ Xk_1 @ GLT))
            
            gamma = (tf.norm(fdx_xk_1) ** 2) / (tf.norm(fdx_xk) ** 2)
            
            Zk_1 = (gamma * Zk) - fdx_xk_1
            
            Xk = Xk_1
            Zk = Zk_1
        
        global test_list
        test_list.append(Xk_1.numpy())
        
        return tf.reshape(Xk_1, [1, N, T])

In [15]:
no_of_psi = 10
Laplacian = adj_to_laplacian(Adj_train)

# sensing_ratio = [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5]
sensing_ratio = (1 + np.arange(80)) / 100.0

reg_list = [1.0e-2] * len(sensing_ratio)
loops_list = [50] * len(sensing_ratio)

all_psi = []

mse_known_list = []
mse_unknown_list = []

learned_variables = []

lr_decay_factor = 1.05

In [16]:
for i_sen, rem in enumerate(sensing_ratio):
    
    print(f"\n\n###########################################################################################")
    print(f"################################### Sensing Ratio: {rem * 100}% ###################################")
    print(f"###########################################################################################\n\n")
    
    if i_sen == 0:
        M = int(rem * T)
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2*T])
    
        for i in range(no_of_psi):
            psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
            np.random.shuffle(psi_k)
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
    
    else:
        previous_psi = all_psi[-no_of_psi : ]
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])
        
        rem_diff = int(N * (rem - sensing_ratio[i_sen - 1]) * T)
        
        for i in range(no_of_psi):
            psi_k_tmp = np.array(previous_psi[i]).flatten()
            ones_idx = np.where(psi_k_tmp == 1)
            
            ones_to_zero_idx = np.random.choice(ones_idx[0], size = rem_diff)
            psi_k_tmp[ones_to_zero_idx] = 0
            psi_k = psi_k_tmp
            
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
        
    reg_init = reg_list[i_sen]
    loops_init = loops_list[i_sen]
    
    sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 1.0e-3, momentum = 0.0)
    
    eldar_model = EldarModel(Laplacian, reg_init, loops_init, d, name = 'EM')
    eldar_model.compile(optimizer = sgd_optimizer, loss = costfunc, 
                        metrics = [mse_known, mse_unknown], run_eagerly = True)
    eldar_model.fit(X_train_concatenated, X_train_missing, epochs = 100, 
                    callbacks = [lr_scheduler], batch_size = 1)
    eldar_model.summary()
    
    all_variables = eldar_model.variables
    
    graph_vars = [i.numpy()[0, 0] for i in all_variables]
    learned_variables.append(graph_vars)
    
    print(f"Learned Variables: {graph_vars}")



###########################################################################################
################################### Sensing Ratio: 1.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 5s 543ms/step - loss: 270.9849 - mse_known: 0.0274 - mse_unknown: 0.0858 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 5s 546ms/step - loss: 270.8474 - mse_known: 0.0274 - mse_unknown: 0.0858 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 5s 548ms/step - loss: 270.7093 - mse_known: 0.0273 - mse_unknown: 0.0858 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 5s 527ms/step - loss: 270.5705 - mse_known: 0.0273 - mse_unknown: 0.0858 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 5s 542ms/step - loss: 270.4311 - mse_known: 0.0273 - mse_unknown: 0.0858 - lr: 0.0010
Epoch 6/100
10/10 [======

Learned Variables: [9.658646, 9.565878, 9.368584, 8.959705]


###########################################################################################
################################### Sensing Ratio: 2.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 863ms/step - loss: 266.9307 - mse_known: 0.0272 - mse_unknown: 0.0825 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 882ms/step - loss: 266.7961 - mse_known: 0.0272 - mse_unknown: 0.0825 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 12s 1s/step - loss: 266.6611 - mse_known: 0.0272 - mse_unknown: 0.0825 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 8s 829ms/step - loss: 266.5252 - mse_known: 0.0272 - mse_unknown: 0.0825 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 857ms/step - loss: 266.3889 - mse_known: 0.0272 - 

Learned Variables: [9.659272, 9.5683975, 9.374901, 8.975416]


###########################################################################################
################################### Sensing Ratio: 3.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 8s 829ms/step - loss: 263.1986 - mse_known: 0.0271 - mse_unknown: 0.0796 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 8s 854ms/step - loss: 263.0673 - mse_known: 0.0271 - mse_unknown: 0.0796 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 8s 826ms/step - loss: 262.9355 - mse_known: 0.0271 - mse_unknown: 0.0796 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 8s 802ms/step - loss: 262.8032 - mse_known: 0.0271 - mse_unknown: 0.0796 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 8s 826ms/step - loss: 262.6702 - mse_known: 0.0271

Learned Variables: [9.65968, 9.570937, 9.381687, 8.992495]


###########################################################################################
################################### Sensing Ratio: 4.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 8s 819ms/step - loss: 259.2486 - mse_known: 0.0270 - mse_unknown: 0.0809 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 858ms/step - loss: 259.1210 - mse_known: 0.0270 - mse_unknown: 0.0809 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 8s 807ms/step - loss: 258.9929 - mse_known: 0.0270 - mse_unknown: 0.0809 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 8s 809ms/step - loss: 258.8642 - mse_known: 0.0270 - mse_unknown: 0.0809 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 8s 805ms/step - loss: 258.7349 - mse_known: 0.0269 -

Learned Variables: [9.660837, 9.574307, 9.389307, 9.010151]


###########################################################################################
################################### Sensing Ratio: 5.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 10s 982ms/step - loss: 255.5720 - mse_known: 0.0269 - mse_unknown: 0.0807 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 11s 1s/step - loss: 255.4477 - mse_known: 0.0269 - mse_unknown: 0.0807 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 888ms/step - loss: 255.3228 - mse_known: 0.0269 - mse_unknown: 0.0806 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 8s 832ms/step - loss: 255.1975 - mse_known: 0.0269 - mse_unknown: 0.0806 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 8s 842ms/step - loss: 255.0717 - mse_known: 0.0268 -

...

Learned Variables: [9.879027, 9.894544, 9.8697605, 9.742248]


###########################################################################################
################################### Sensing Ratio: 76.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 867ms/step - loss: 25.0109 - mse_known: 0.0101 - mse_unknown: 0.1741 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 895ms/step - loss: 25.0031 - mse_known: 0.0101 - mse_unknown: 0.1741 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 886ms/step - loss: 24.9949 - mse_known: 0.0101 - mse_unknown: 0.1741 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 881ms/step - loss: 24.9869 - mse_known: 0.0100 - mse_unknown: 0.1741 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 875ms/step - loss: 24.9788 - mse_known: 0.0100 - m

Learned Variables: [9.885138, 9.900411, 9.877063, 9.755452]


###########################################################################################
################################### Sensing Ratio: 77.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 890ms/step - loss: 23.2719 - mse_known: 0.0097 - mse_unknown: 0.1767 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 901ms/step - loss: 23.2648 - mse_known: 0.0097 - mse_unknown: 0.1767 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 901ms/step - loss: 23.2575 - mse_known: 0.0097 - mse_unknown: 0.1767 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 915ms/step - loss: 23.2503 - mse_known: 0.0097 - mse_unknown: 0.1767 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 903ms/step - loss: 23.2430 - mse_known: 0.0097 - ms

Learned Variables: [9.891067, 9.906148, 9.8842535, 9.768144]


###########################################################################################
################################### Sensing Ratio: 78.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 896ms/step - loss: 21.3405 - mse_known: 0.0093 - mse_unknown: 0.1798 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 935ms/step - loss: 21.3341 - mse_known: 0.0093 - mse_unknown: 0.1798 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 911ms/step - loss: 21.3276 - mse_known: 0.0093 - mse_unknown: 0.1798 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 913ms/step - loss: 21.3211 - mse_known: 0.0093 - mse_unknown: 0.1798 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 880ms/step - loss: 21.3148 - mse_known: 0.0093 - m

Learned Variables: [9.897797, 9.912811, 9.891989, 9.779464]


###########################################################################################
################################### Sensing Ratio: 79.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 909ms/step - loss: 19.5961 - mse_known: 0.0089 - mse_unknown: 0.1819 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 917ms/step - loss: 19.5904 - mse_known: 0.0089 - mse_unknown: 0.1819 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 12s 1s/step - loss: 19.5845 - mse_known: 0.0089 - mse_unknown: 0.1819 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 13s 1s/step - loss: 19.5788 - mse_known: 0.0089 - mse_unknown: 0.1820 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 13s 1s/step - loss: 19.5730 - mse_known: 0.0089 - mse_unkn

Learned Variables: [9.903782, 9.918504, 9.898597, 9.789755]


###########################################################################################
################################### Sensing Ratio: 80.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 9s 890ms/step - loss: 17.9708 - mse_known: 0.0086 - mse_unknown: 0.1843 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 9s 876ms/step - loss: 17.9658 - mse_known: 0.0086 - mse_unknown: 0.1843 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 9s 909ms/step - loss: 17.9606 - mse_known: 0.0086 - mse_unknown: 0.1843 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 9s 902ms/step - loss: 17.9555 - mse_known: 0.0086 - mse_unknown: 0.1843 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 9s 901ms/step - loss: 17.9504 - mse_known: 0.0086 - ms

In [17]:
for i, rem in enumerate(sensing_ratio):
    print(f"For {int(100 * rem)}% sensing ratio: {learned_variables[i]}")

For 1% sensing ratio: [9.658646, 9.565878, 9.368584, 8.959705]
For 2% sensing ratio: [9.659272, 9.5683975, 9.374901, 8.975416]
For 3% sensing ratio: [9.65968, 9.570937, 9.381687, 8.992495]
For 4% sensing ratio: [9.660837, 9.574307, 9.389307, 9.010151]
For 5% sensing ratio: [9.662014, 9.577462, 9.396418, 9.026382]
For 6% sensing ratio: [9.663235, 9.580696, 9.403261, 9.041059]
For 7% sensing ratio: [9.664699, 9.584353, 9.411023, 9.058115]
For 8% sensing ratio: [9.665597, 9.587335, 9.417844, 9.073103]
For 9% sensing ratio: [9.666826, 9.590578, 9.424711, 9.087473]
For 10% sensing ratio: [9.668728, 9.594749, 9.432751, 9.103356]
For 11% sensing ratio: [9.670119, 9.598477, 9.440499, 9.1191225]
For 12% sensing ratio: [9.670863, 9.601299, 9.446913, 9.1328945]
For 13% sensing ratio: [9.672374, 9.604914, 9.454144, 9.147376]
For 14% sensing ratio: [9.674494, 9.6091, 9.462048, 9.162695]
For 15% sensing ratio: [9.675108, 9.611743, 9.467896, 9.174989]
For 16% sensing ratio: [9.676333, 9.61514, 9.4749

In [18]:
np.save('./Outputs/eldar_GT_mse_known.npy', mse_known_list)
np.save('./Outputs/eldar_GT_mse_unknown.npy', mse_unknown_list)